# Video Analysis

In [10]:
import pytube
# Indie/Pop/Folk Compilation - December 2021 (2-Hour Playlist)
url = 'https://www.youtube.com/watch?v=6no3uMeMIr8'

In [ ]:
youtube = pytube.YouTube(url)
streams = youtube.streams

In [33]:
streams.get_audio_only()

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">

In [46]:
mp4 = streams.filter(file_extension='mp4',progressive=True).order_by('resolution')
mp4.first()

<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">

In [47]:
mp4.first().download('downloads')

'd:\\devel\\music-generator\\downloads\\IndiePopFolk Compilation - December 2021 (2-Hour Playlist).mp4'

In [11]:
audio = streams.filter(type="audio")
audio

[<Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">, <Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">, <Stream: itag="249" mime_type="audio/webm" abr="50kbps" acodec="opus" progressive="False" type="audio">, <Stream: itag="250" mime_type="audio/webm" abr="70kbps" acodec="opus" progressive="False" type="audio">, <Stream: itag="251" mime_type="audio/webm" abr="160kbps" acodec="opus" progressive="False" type="audio">]

In [13]:
[s.abr for s in audio]

['48kbps', '128kbps', '50kbps', '70kbps', '160kbps']

In [22]:
[s.abr + " - " + s.type + " (" + str(round(s.filesize/2**10/2**10)) + " MB)" for s in audio]

['48kbps - audio (45 MB)',
 '128kbps - audio (119 MB)',
 '50kbps - audio (48 MB)',
 '70kbps - audio (63 MB)',
 '160kbps - audio (124 MB)']

In [23]:
s = streams.filter(type = "audio").order_by('abr').desc().first()
s

<Stream: itag="251" mime_type="audio/webm" abr="160kbps" acodec="opus" progressive="False" type="audio">

In [25]:
s.filesize
# translate to kB
s.filesize/2**10
# translate to MB
s.filesize/2**10/2**10

124.11291313171387

In [26]:
s.download('downloads')

'd:\\devel\\music-generator\\downloads\\IndiePopFolk Compilation - December 2021 (2-Hour Playlist).webm'

In [37]:
title = s.title
title

'Indie/Pop/Folk Compilation - December 2021 (2-Hour Playlist)'

## Select video sequence 

In [48]:
import os
dl = os.listdir("downloads")
dl

['IndiePopFolk Compilation - December 2021 (2-Hour Playlist).mp4',
 'IndiePopFolk Compilation - December 2021 (2-Hour Playlist).webm']

In [51]:
import moviepy.editor as mp
video = mp.VideoFileClip(os.path.join("downloads",dl[0]))
video.audio.write_audiofile(os.path.join("downloads","music.mp3"))


chunk:   0%|          | 26/169954 [01:28<2:12:42, 21.34it/s, now=None]

MoviePy - Writing audio in downloads\music.mp3


MoviePy - Done.


### Manipulate video duration format

https://realpython.com/python-time-module/

In [52]:
d=int(video.duration)
d

7707

In [55]:
from datetime import date, datetime, time, timedelta

dt = datetime.combine(date.today(), time(0, 0)) + timedelta(seconds=d)
dt.time()

datetime.time(2, 8, 27)

In [63]:
print("%02d:%02d:%02d" % (dt.hour,dt.minute,dt.second))


02:08:27


In [65]:
from datetime import date, datetime, time, timedelta

def format_time(d):
    
    dt = datetime.combine(date.today(), time(0, 0)) + timedelta(seconds=d)
    return "%02d:%02d:%02d" % (dt.hour,dt.minute,dt.second)

format_time(d)

'02:08:27'

## Cut the playlist into musics chunks

In [1]:
import urllib.request
url = 'https://www.youtube.com/watch?v=6no3uMeMIr8'
with urllib.request.urlopen(url) as response:
   html = response.read()

In [19]:
import urllib
urllib.request.urlretrieve(url,"downloads/page.html")

('downloads/page.html', <http.client.HTTPMessage at 0x1e6449ccaf0>)

In [24]:
import re
with open("downloads/page.html", encoding="utf8") as f:
    text = f.read()
m = re.search('Tracklist:(.+?)Find alexrainbirdMusic on', text)
m.group(0)[0:100]

'Tracklist:\\n0:00 The Invention of Flight - ‘You’\\nhttps://spoti.fi/3DGQkCa\\nhttps://www.instagram.co'

In [26]:
with open("downloads/tracklist.txt","w", encoding="utf8") as f:
    f.write(m.group(0))

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [14]:
# soup.find_all("a",{"class":"yt-simple-endpoint style-scope yt-formatted-string"})
soup.find("div",{"id":"description"})

In [18]:
len(soup.find_all("a"))

14

In [10]:
xpathselector = '//*[@id="description"]/yt-formatted-string/a[6]'

In [ ]:
from lxml import etree
htmlparser = etree.HTMLParser()
tree = etree.parse(html, htmlparser)
tree.xpath(xpathselector)